In [ ]:
import csv
from therapy import PROJECT_ROOT
from therapy.query import QueryHandler
from therapy.schemas import MatchType

cc0_file = PROJECT_ROOT / 'data' / 'drugbank_cc0' / 'drugbank_cc0_20210414.csv'
query_handler = QueryHandler(db_url='http://localhost:8000')

In [ ]:
with open(cc0_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header
    for row in reader:
        alias = row[2]
        try:
            response = query_handler.search_sources(query_str=alias, incl='drugbank')
        except TypeError:
            print('****')
            print(f'Error: {alias}')
        if response['source_matches'][0]['match_type'] != MatchType.LABEL:
            print('****')
            print(alias)
            print(response['source_matches'][0]['match_type'])

In [ ]:
with open(cc0_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header
    for row in reader:
        cc0_label = row[2]
        cc0_aliases = {a for a in row[1].split(' | ') + row[5].split(' | ') if a}
        try:
            response = query_handler.search_sources(query_str=cc0_label, incl='drugbank')
            if response['source_matches'][0]['match_type'] == MatchType.LABEL:
                response_aliases = {a for a in response['source_matches'][0]['records'][0].aliases if a}
                diff_a = cc0_aliases.difference(response_aliases)
                diff_b = response_aliases.difference(cc0_aliases)
                if diff_a or diff_b:
                    print('******')
                    print(cc0_label)
                    if diff_a:
                        print(f"in cc0: {diff_a}")
                    if diff_b:
                        print(f"in db: {diff_b}")
        except TypeError:
            print('****')
            print(f'Error: {alias}')